In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
%matplotlib inline

from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Dropout, Layer, Activation, Reshape, Permute
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization

import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
import h5py
import random

from helper import *
from segnet_model import *

from distutils.version import LooseVersion

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

Using TensorFlow backend.


TensorFlow Version: 1.0.0


In [2]:
download_data()

In [3]:
segnet_basic = get_segnet_basic()

In [4]:
segnet_basic.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_1 (Layer)              (None, 600, 800, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 602, 802, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 600, 800, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 600, 800, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 600, 800, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 300, 400, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 302, 402, 64)      0         
__________

In [5]:
#segnet_basic.summary()
segnet_basic.compile(loss="categorical_crossentropy", optimizer='adadelta', metrics=["accuracy"])

In [6]:
# There are 1000 images. Each one flip once, that product 2000 image.
samples = [i for i in range(2000)]
shuffle(samples)

train_samples, validation_samples = train_test_split(samples, test_size=0.25)
validation_samples, test_samples = train_test_split(validation_samples, test_size=0.3)

In [7]:
#train_data, train_label = prep_data(test_samples)
#validate_data, validate_label = prep_data(validation_samples)
#test_data, test_label = prep_data(test_samples)
#epochs = 2
#batch_size = 10

#result = segnet_basic.fit(train_data, train_label, batch_size=batch_size, 
#                          epochs=epochs, verbose=1, shuffle=True)

In [8]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)
test_generator = generator(test_samples, batch_size=32)

In [11]:
segnet_basic.fit_generator(train_generator, steps_per_epoch= len(train_samples),
                    validation_data=validation_generator, validation_steps=len(validation_samples), epochs=15)

Epoch 1/1
  1/700 [..............................] - ETA: 59098s - loss: 3.6326 - acc: 0.0600

KeyboardInterrupt: 

In [ ]:
scores = segnet_basic.evaluate_generator(test_generator, len(test_samples))
print("Accuracy = ", scores)

In [ ]:
import h5py
from keras.models import model_from_json

segnet_basic.save('model_with_weight.h5')

segnet_basic.save_weights("model_weight.hdf5")
print("sava weight done..")    

json_string = segnet_basic.to_json()

with open("model.json", "w") as json_file:
    json_file.write(json_string)